# Use selected training set to create input for fitting code

In [1]:
# Compatibility with Python 3
from __future__ import (absolute_import, division, print_function)

try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

# Basic Tools
import numpy as np
import copy
import pickle
from astropy.table import Table
from astropy.io import fits
import corner
import matplotlib.pyplot as plt
from scipy.io import readsav
from scipy.ndimage.filters import convolve
import time

# The Cannon
# import thecannon as tc

In [2]:
galah_elements = [
    'Li','C','O',
    'Na','Mg','Al','Si',
    'K','Ca','Sc','Ti','V','Cr','Mn','Co','Ni','Cu','Zn',
    'Rb','Sr','Y','Zr','Mo','Ru',
    'Ba','La','Ce','Nd','Sm','Eu'
]

In [4]:
synthesis_files = '/Users/svenbuder/GALAH_DR4/spectrum_grids/marcs2014/specout/'
wavelength_file = 'training_sets/solar_twin_5steps_training_marcs2014_wavelength.pickle'

training_set = Table.read('../../spectrum_grids/marcs2014/marcs2014_scaledsolar_210720.fits')
training_set['INDEX'][np.where((training_set['TEFF'] == 5250) & (training_set['FEH'] == -3))[0]]
# #2159,2216,3967

example_0p5 = readsav(synthesis_files+'marcs2014_scaledsolar_210720_2159_ccd1_smod_sint.sav').results[0]
example_2p5 = readsav(synthesis_files+'marcs2014_scaledsolar_210720_2216_ccd1_smod_sint.sav').results[0]
example_4p5 = readsav(synthesis_files+'marcs2014_scaledsolar_210720_3967_ccd1_smod_sint.sav').results[0]

In [16]:
def gaussbroad(w, s, hwhm):
    """
    Smooths a spectrum by convolution with a gaussian of specified hwhm.
    Parameters
    -------
    w : array[n]
        wavelength scale of spectrum to be smoothed
    s : array[n]
        spectrum to be smoothed
    hwhm : float
        half width at half maximum of smoothing gaussian.
    Returns
    -------
    sout: array[n]
        the gaussian-smoothed spectrum.
    """
    """
    History
    --------
        Dec-90 GB,GM
            Rewrote with fourier convolution algorithm.
        Jul-91 AL
            Translated from ANA to IDL.
        22-Sep-91 JAV
            Relaxed constant dispersion check# vectorized, 50% faster.
        05-Jul-92 JAV
            Converted to function, handle nonpositive hwhm.
        Oct-18 AW
            Python version
    """

    # Warn user if hwhm is negative.
    if hwhm < 0:
        logger.warning("Forcing negative smoothing width to zero.")

    # Return input argument if half-width is nonpositive.
    if hwhm <= 0:
        return s  # true: no broadening

    # Calculate (uniform) dispersion.
    nw = len(w)  ## points in spectrum
    wrange = w[-1] - w[0]
    dw = wrange / (nw - 1)  # wavelength change per pixel

    # Make smoothing gaussian# extend to 4 sigma.
    # 4.0 / sqrt(2.0*alog(2.0)) = 3.3972872 and sqrt(alog(2.0))=0.83255461
    # sqrt(alog(2.0)/pi)=0.46971864 (*1.0000632 to correct for >4 sigma wings)
    if hwhm >= 5 * wrange:
        return np.full(nw, np.sum(s) / nw)
    nhalf = int(3.3972872 * hwhm / dw)  ## points in half gaussian
    ng = 2 * nhalf + 1  ## points in gaussian (odd!)
    wg = dw * (
        np.arange(ng, dtype=float) - (ng - 1) / 2
    )  # wavelength scale of gaussian
    xg = (0.83255461 / hwhm) * wg  # convenient absisca
    gpro = (0.46974832 * dw / hwhm) * np.exp(-xg * xg)  # unit area gaussian w/ FWHM
    gpro = gpro / np.sum(gpro)

    # Pad spectrum ends to minimize impact of Fourier ringing.
    sout = convolve(s, gpro, mode="nearest")

    return sout

In [17]:
def apply_gauss_broad(wave, smod, ipres=30000, debug=True):
    ccd_time = time.perf_counter()
    # Apply Gaussian Instrument Broadening
    if ipres == 0.0:
        hwhm = 0
    else:
        hwhm = 0.5 * wave[0] / ipres
    if hwhm > 0: smod = gaussbroad(wave, smod, hwhm)

    if debug:
        ccd_time = time.perf_counter() - ccd_time
        print('Gaussbroad time: ',ccd_time)

    return(smod)

In [18]:
def integrate_flux(mu, inten, deltav, vsini, vrt, osamp=1):
    """
    Produces a flux profile by integrating intensity profiles (sampled
    at various mu angles) over the visible stellar surface.
    Intensity profiles are weighted by the fraction of the projected
    stellar surface they represent, apportioning the area between
    adjacent MU points equally. Additional weights (such as those
    used in a Gauss-Legendre quadrature) can not meaningfully be
    used in this scheme.  About twice as many points are required
    with this scheme to achieve the precision of Gauss-Legendre
    quadrature.
    DELTAV, VSINI, and VRT must all be in the same units (e.g. km/s).
    If specified, OSAMP should be a positive integer.
    Parameters
    ----------
    mu : array(float) of size (nmu,)
        cosine of the angle between the outward normal and
        the line of sight for each intensity spectrum in INTEN.
    inten : array(float) of size(nmu, npts)
        intensity spectra at specified values of MU.
    deltav : float
        velocity spacing between adjacent spectrum points
        in INTEN (same units as VSINI and VRT).
    vsini : float
        maximum radial velocity, due to solid-body rotation.
    vrt : float
        radial-tangential macroturbulence parameter, i.e.
        np.sqrt(2) times the standard deviation of a Gaussian distribution
        of turbulent velocities. The same distribution function describes
        the radial motions of one component and the tangential motions of
        a second component. Each component covers half the stellar surface.
        See 'The Observation and Analysis of Stellar Photospheres', Gray.
    osamp : int, optional
        internal oversampling factor for convolutions.
        By default convolutions are done using the input points (OSAMP=1),
        but when OSAMP is set to higher integer values, the input spectra
        are first oversampled by cubic spline interpolation.
    Returns
    -------
    value : array(float) of size (npts,)
        Disk integrated flux profile.
    Note
    ------------
        If you use this algorithm in work that you publish, please cite
        Valenti & Anderson 1996, PASP, currently in preparation.
    """
    """
    History
    -----------
    Feb-88  GM
        Created ANA version.
    13-Oct-92 JAV
        Adapted from G. Marcy's ANA routi!= of the same name.
    03-Nov-93 JAV
        Switched to annular convolution technique.
    12-Nov-93 JAV
        Fixed bug. Intensity compo!=nts not added when vsini=0.
    14-Jun-94 JAV
        Reformatted for "public" release. Heavily commented.
        Pass deltav instead of 2.998d5/deltav. Added osamp
        keyword. Added rebinning logic at end of routine.
        Changed default osamp from 3 to 1.
    20-Feb-95 JAV
        Added mu as an argument to handle arbitrary mu sampling
        and remove ambiguity in intensity profile ordering.
        Interpret VTURB as np.sqrt(2)*sigma instead of just sigma.
        Replaced call_external with call to spl_{init|interp}.
    03-Apr-95 JAV
        Multiply flux by pi to give observed flux.
    24-Oct-95 JAV
        Force "nmk" padding to be at least 3 pixels.
    18-Dec-95 JAV
        Renamed from dskint() to rtint(). No longer make local
        copy of intensities. Use radial-tangential instead
        of isotropic Gaussian macroturbulence.
    26-Jan-99 JAV
        For NMU=1 and VSINI=0, assume resolved solar surface#
        apply R-T macro, but supress vsini broadening.
    01-Apr-99 GMH
        Use annuli weights, rather than assuming ==ual area.
    07-Mar-12 JAV
        Force vsini and vmac to be scalars.
    """

    # Make local copies of various input variables, which will be altered below.
    # Force vsini and especially vmac to be scalars. Otherwise mu dependence fails.

    if np.size(vsini) > 1:
        vsini = vsini[0]
    if np.size(vrt) > 1:
        vrt = vrt[0]

    # Determine oversampling factor.
    os = round(np.clip(osamp, 1, None))  # force integral value > 1

    # Convert input MU to projected radii, R, of annuli for a star of unit radius
    #  (which is just sine, rather than cosine, of the angle between the outward
    #  normal and the line of sight).
    rmu = np.sqrt(1 - mu ** 2)  # use simple trig identity

    # Sort the projected radii and corresponding intensity spectra into ascending
    #  order (i.e. from disk center to the limb), which is equivalent to sorting
    #  MU in descending order.
    isort = np.argsort(rmu)
    rmu = rmu[isort]  # reorder projected radii
    nmu = np.size(mu)  # number of radii
    if nmu == 1:
        if vsini != 0:
            logger.warning(
                "Vsini is non-zero, but only one projected radius (mu value) is set. No rotational broadening will be performed."
            )
            vsini = 0  # ignore vsini if only 1 mu

    # Calculate projected radii for boundaries of disk integration annuli.  The n+1
    # boundaries are selected such that r(i+1) exactly bisects the area between
    # rmu(i) and rmu(i+1). The in!=rmost boundary, r(0) is set to 0 (disk center)
    # and the outermost boundary, r(nmu) is set to 1 (limb).
    if nmu > 1 or vsini != 0:  # really want disk integration
        r = np.sqrt(
            0.5 * (rmu[:-1] ** 2 + rmu[1:] ** 2)
        )  # area midpoints between rmu
        r = np.concatenate(([0], r, [1]))

        # Calculate integration weights for each disk integration annulus.  The weight
        # is just given by the relative area of each annulus, normalized such that
        # the sum of all weights is unity.  Weights for limb darkening are included
        # explicitly in the intensity profiles, so they aren't needed here.
        wt = r[1:] ** 2 - r[:-1] ** 2  # weights = relative areas
    else:
        wt = np.array([1.0])  # single mu value, full weight

    # Generate index vectors for input and oversampled points. Note that the
    # oversampled indicies are carefully chosen such that every "os" finely
    # sampled points fit exactly into one input bin. This makes it simple to
    # "integrate" the finely sampled points at the end of the routine.
    npts = inten.shape[1]  # number of points
    xpix = np.arange(npts, dtype=float)  # point indices
    nfine = os * npts  # number of oversampled points
    xfine = (0.5 / os) * (
        2 * np.arange(nfine, dtype=float) - os + 1
    )  # oversampled points indices

    # Loop through annuli, constructing and convolving with rotation kernels.

    yfine = np.empty(nfine)  # init oversampled intensities
    flux = np.zeros(nfine)  # init flux vector
    for imu in range(nmu):  # loop thru integration annuli

        #  Use external cubic spline routine (adapted from Numerical Recipes) to make
        #  an oversampled version of the intensity profile for the current annulus.
        ypix = inten[isort[imu]]  # extract intensity profile
        if os == 1:
            # just copy (use) original profile
            yfine = ypix
        else:
            # spline onto fine wavelength scale
            yfine = interp1d(xpix, ypix, kind="cubic")(xfine)

        # Construct the convolution kernel which describes the distribution of
        # rotational velocities present in the current annulus. The distribution has
        # been derived analytically for annuli of arbitrary thickness in a rigidly
        # rotating star. The kernel is constructed in two pieces: o!= piece for
        # radial velocities less than the maximum velocity along the inner edge of
        # the annulus, and one piece for velocities greater than this limit.
        if vsini > 0:
            # nontrivial case
            r1 = r[imu]  # inner edge of annulus
            r2 = r[imu + 1]  # outer edge of annulus
            dv = deltav / os  # oversampled velocity spacing
            maxv = vsini * r2  # maximum velocity in annulus
            nrk = 2 * int(maxv / dv) + 3  ## oversampled kernel point
            # velocity scale for kernel
            v = dv * (np.arange(nrk, dtype=float) - ((nrk - 1) / 2))
            rkern = np.zeros(nrk)  # init rotational kernel
            j1 = np.abs(v) < vsini * r1  # low velocity points
            rkern[j1] = np.sqrt((vsini * r2) ** 2 - v[j1] ** 2) - np.sqrt(
                (vsini * r1) ** 2 - v[j1] ** 2
            )  # generate distribution

            j2 = (np.abs(v) >= vsini * r1) & (np.abs(v) <= vsini * r2)
            rkern[j2] = np.sqrt(
                (vsini * r2) ** 2 - v[j2] ** 2
            )  # generate distribution

            rkern = rkern / np.sum(rkern)  # normalize kernel

            # Convolve the intensity profile with the rotational velocity kernel for this
            # annulus. Pad each end of the profile with as many points as are in the
            # convolution kernel. This reduces Fourier ringing. The convolution may also
            # be do!= with a routi!= called "externally" from IDL, which efficiently
            # shifts and adds.
            if nrk > 3:
                yfine = convolve(yfine, rkern, mode="nearest")

        # Calculate projected sigma for radial and tangential velocity distributions.
        muval = mu[isort[imu]]  # current value of mu
        sigma = os * vrt / np.sqrt(2) / deltav  # standard deviation in points
        sigr = sigma * muval  # reduce by current mu value
        sigt = sigma * np.sqrt(1.0 - muval ** 2)  # reduce by np.sqrt(1-mu**2)

        # Figure out how many points to use in macroturbulence kernel.
        nmk = int(10 * sigma)
        nmk = np.clip(nmk, 3, (nfine - 3) // 2)

        # Construct radial macroturbulence kernel with a sigma of mu*VRT/np.sqrt(2).
        if sigr > 0:
            xarg = np.linspace(-nmk, nmk, 2 * nmk + 1) / sigr
            xarg = np.clip(-0.5 * xarg ** 2, -20, None)
            mrkern = np.exp(xarg)  # compute the gaussian
            mrkern = mrkern / np.sum(mrkern)  # normalize the profile
        else:
            mrkern = np.zeros(2 * nmk + 1)  # init with 0d0
            mrkern[nmk] = 1.0  # delta function

        # Construct tangential kernel with a sigma of np.sqrt(1-mu**2)*VRT/np.sqrt(2).
        if sigt > 0:
            xarg = np.linspace(-nmk, nmk, 2 * nmk + 1) / sigt
            xarg = np.clip(-0.5 * xarg ** 2, -20, None)
            mtkern = np.exp(xarg)  # compute the gaussian
            mtkern = mtkern / np.sum(mtkern)  # normalize the profile
        else:
            mtkern = np.zeros(2 * nmk + 1)  # init with 0d0
            mtkern[nmk] = 1.0  # delta function

        # Sum the radial and tangential components, weighted by surface area.
        area_r = 0.5  # assume equal areas
        area_t = 0.5  # ar+at must equal 1
        mkern = area_r * mrkern + area_t * mtkern  # add both components

        # Convolve the total flux profiles, again padding the spectrum on both ends to
        # protect against Fourier ringing.
        yfine = convolve(
            yfine, mkern, mode="nearest"
        )  # add the padding and convolve

        # Add contribution from current annulus to the running total.
        flux = flux + wt[imu] * yfine  # add profile to running total

    flux = np.reshape(flux, (npts, os))  # convert to an array
    flux = np.pi * np.sum(flux, axis=1) / os  # sum, normalize
    return flux

In [205]:
def broaden_spectrum(wint_seg, sint_seg, wave_seg, cmod_seg, vsini=0, vmac=0, debug=False):

    nw = len(wint_seg)
    clight = 299792.5
    mu = (np.sqrt(0.5*(2*np.arange(7)+1)/np.float(7)))[::-1]
    nmu = 7
    wmid = 0.5 * (wint_seg[nw-1] + wint_seg[0])
    wspan = wint_seg[nw-1] - wint_seg[0]
    jmin = np.argmin(wint_seg[1:nw-1] - wint_seg[0:nw-2])
    vstep1 = min(wint_seg[1:nw-1] - wint_seg[0:nw-2])
    vstep2 = 0.1 * wspan / (nw-1) / wmid * clight
    vstep3 = 0.05
    vstep = np.max([vstep1,vstep2,vstep3])

    # Generate model wavelength scale X, with uniform wavelength step.
    nx = int(np.floor(np.log10(wint_seg[nw-1] / wint_seg[0])/ np.log10(1.0+vstep / clight))+1)
    if nx % 2 == 0: nx += 1
    resol_out = 1.0/((wint_seg[nw-1] / wint_seg[0])**(1.0/(nx-1.0))-1.0)
    vstep = clight / resol_out
    x_seg = wint_seg[0] * (1.0 + 1.0 / resol_out)**np.arange(nx)

    # Interpolate intensity spectra onto new model wavelength scale.  
    yi_seg = np.empty((nmu, nx))

    for imu in range(nmu):
        yi_seg[imu] = np.interp(x_seg, wint_seg, sint_seg[imu])

    y_seg = integrate_flux(mu, yi_seg, vstep, np.abs(vsini), np.abs(vmac))

    dispersion = vstep1
    wave_equi = np.arange(x_seg[0],x_seg[-1]+dispersion,dispersion)

    c_seg = np.interp(wave_equi,wave_seg,cmod_seg)
    y_seg = np.interp(wave_equi,x_seg,y_seg)

    if debug:
        print(vstep1,len(wave_equi))
    
    return(wave_equi,y_seg/c_seg)

In [207]:
# Define a common wavelength grid that we will use for all spectra
wavelengths_for_each_ccd = dict()
for ccd in [1,2,3,4]:
    example_ccd = readsav(synthesis_files+'marcs2014_scaledsolar_210720_0_ccd'+str(ccd)+'_smod_sint.sav').results[0]
    example_wave, example_flux = broaden_spectrum(example_ccd.wint, example_ccd.sint, example_ccd.wave, example_ccd.cmod, vsini=0, vmac=0)
    wavelengths_for_each_ccd['CCD'+str(ccd)] = example_wave

wavelength_array = np.concatenate(([wavelengths_for_each_ccd['CCD'+ccd] for ccd in ['1','2','3','4']]))

wavelength_file_opener = open(wavelength_file,'wb')
pickle.dump((wavelength_array),wavelength_file_opener)
wavelength_file_opener.close()

In [208]:
# Let's create a matrix that we will later fill with the normalised flux values
normalized_flux = np.ones((np.shape(training_set)[0],np.shape(wavelength_array)[0]))
normalized_ivar = np.ones((np.shape(training_set)[0],np.shape(wavelength_array)[0]))

In [10]:
def load_normalised_spectra(index, wavelengths_for_each_ccd, vsini=0.0, spectrum_path = '/Users/svenbuder/galah_solar_twins/dr3_spectra/hermes'):
    
    # For each stars, there are 4 spectra for the 4 different CCDs.
    # We will interpolate the fluxes and uncertainties/inverse variances onto a common grid
    normalised_flux_for_index = []
    normalised_ivar_for_index = []
    # For that we first interpolate over the individual CCDs
    #for ccd in ['3','4']:
    for ccd in ['1','2','3','4']:
    #for ccd in ['2','3']:
        
        #synthetic_spectrum = readsav(synthesis_files+'/solar_twin_grid_210831_'+str(index)+'_ccd'+ccd+'_smod_sint.sav').results[0]
        synthetic_spectrum = readsav(synthesis_files+'/marcs2014_scaledsolar_210720_'+str(index)+'_ccd'+ccd+'_smod_sint.sav').results[0]

        wave_broadened,flux_broadened = broaden_spectrum(
            synthetic_spectrum.wint,
            synthetic_spectrum.sint,
            synthetic_spectrum.wave,
            synthetic_spectrum.cmod,
            vsini=vsini)
        
        interpolated_broadened_smod = np.interp(wavelengths_for_each_ccd['CCD'+ccd],synthetic_spectrum.wave,broadened_smod)
        
        normalised_flux_for_index.append(interpolated_broadened_smod)
        # We use synthetic spectra, so SNR == infinity
        # Let's assume SNR = 1000, so std == 0.001, so 1/var = 1/std**2 = 1,000,000.
        normalised_ivar_for_index.append(1000000.*np.ones_like(interpolated_broadened_smod))
        
    normalised_flux_for_index = np.concatenate((normalised_flux_for_index))
    normalised_ivar_for_index = np.concatenate((normalised_ivar_for_index))
    
    return(normalised_flux_for_index,normalised_ivar_for_index)

In [11]:
def populate_normalised_flux_and_ivar_matrix(training_set, matrix_index, wavelengths_for_each_ccd):
    index = training_set['INDEX'][matrix_index]
    #try:
    normalised_flux_for_index, normalised_ivar_for_index = load_normalised_spectra(index,wavelengths_for_each_ccd=wavelengths_for_each_ccd)
    #except:
    #    print('Failed to load spectrum for index '+str(matrix_index)+', that is, index '+str(index))
    normalized_flux[matrix_index] = normalised_flux_for_index
    normalized_ivar[matrix_index] = normalised_ivar_for_index

In [15]:
training_set_all = Table.read('../../spectrum_grids/marcs2014/marcs2014_scaledsolar_210720.fits')

# 2160 models with
# teff 2500..(100/250)..8000 K including 5 at a time
# logg = -0.5..(0.5)..5.0/5.5 dex
# feh = -3.0..(0.5/0.25)..1.0 dex

teff_points = np.unique(training_set_all['TEFF'])
logg_points = np.unique(training_set_all['LOGG'])
fe_h_points = np.unique(training_set_all['FEH'])

cannon_index = []
cannon_teff = []
cannon_logg = []
cannon_feh = []

i = 0
for teff in teff_points[2:-3]:
    for logg in logg_points[1:-2]:
        for fe_h in fe_h_points[4:-3]:
            teff_i = np.where(teff==teff_points)[0][0]
            logg_i = np.where(logg==logg_points)[0][0]
            fe_h_i = np.where(fe_h==fe_h_points)[0][0]
            
            grid = (
                    (training_set_all['TEFF'] >= teff_points[teff_i-2]) &
                    (training_set_all['TEFF'] <= teff_points[teff_i+2]) &
                    (training_set_all['LOGG'] >= logg_points[logg_i-1]) &
                    (training_set_all['LOGG'] <= logg_points[logg_i+1]) &
                    (training_set_all['FEH'] >= fe_h_points[fe_h_i-2]) &
                    (training_set_all['FEH'] <= fe_h_points[fe_h_i+2]) &
                    ((training_set_all['INDEX'] < 107) | (training_set_all['INDEX'] > 109)) &
                    ((training_set_all['INDEX'] < 119) | (training_set_all['INDEX'] > 122)) &
                    ((training_set_all['INDEX'] < 131) | (training_set_all['INDEX'] > 134)) &
                    ((training_set_all['INDEX'] < 143) | (training_set_all['INDEX'] > 146)) &
                    ((training_set_all['INDEX'] < 203) | (training_set_all['INDEX'] > 205)) &
                    ((training_set_all['INDEX'] < 212) | (training_set_all['INDEX'] > 213)) &
                    ((training_set_all['INDEX'] < 3980) | (training_set_all['INDEX'] > 3980)) 
                
                )
            
            grid_size = len(training_set_all[grid])
            
            if grid_size > 10:

                cannon_index.append(i)
                cannon_teff.append(teff)
                cannon_logg.append(logg)
                cannon_feh.append(fe_h)

                #if (teff == 5500) & (logg == 4.5) & (fe_h == 0.00):
                #    print(i)
                
                if i in [1813]:#550,1000,1206,1656,1660,1664,1668,1669,1670,1671,1672,1673,1744,2000,2050]:
                    
                    
                    
                    #if (grid_size > 0) & (grid_size < 50):
                    print(teff,logg,fe_h,grid_size)

                    training_set = copy.deepcopy(training_set_all[grid])
                    training_set.write('training_sets/subgrid_'+str(i)+'_training_set_marcs2014.fits',overwrite=True)

                    flux_ivar_file = 'training_sets/subgrid_'+str(i)+'_training_marcs2014_flux_ivar.pickle'

                    # Let's create a matrix that we will later fill with the normalised flux values
                    normalized_flux = np.ones((np.shape(training_set)[0],np.shape(wavelength_array)[0]))
                    normalized_ivar = np.ones((np.shape(training_set)[0],np.shape(wavelength_array)[0]))

                    [populate_normalised_flux_and_ivar_matrix(training_set, matrix_index=index, wavelengths_for_each_ccd=wavelengths_for_each_ccd) for index in range(np.shape(training_set)[0])];

                    flux_ivar_file_opener = open(flux_ivar_file,'wb')
                    pickle.dump((normalized_flux,normalized_ivar),flux_ivar_file_opener)
                    flux_ivar_file_opener.close()

                i+= 1

cannon_grid = Table()
cannon_grid['index'] = np.array(cannon_index)
cannon_grid['teff'] = np.array(cannon_teff)
cannon_grid['logg'] = np.array(cannon_logg)
cannon_grid['fe_h'] = np.array(cannon_feh)
cannon_grid.write('Cannon_subgrid.fits',overwrite=True)

6250.0 4.5 0.0 75
